# Skill Estimation Project (using pyGM)

In [43]:
import pyGM as gm
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline  

### Games and Outcomes

In [44]:
import csv
global players
players = 0
player_dict = dict()
with open('train-57922.csv', newline='') as csv_file: # replace csv file with the training dataset being used
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if str(row[1]) not in player_dict.values():
            player_dict[players] = str(row[1])
            players += 1
print(player_dict)

{0: 'MC', 1: 'Life', 2: 'sOs', 3: 'Polt', 4: 'TY', 5: 'INnoVation', 6: 'Mvp', 7: 'ByuN', 8: 'MMA', 9: 'PartinG', 10: 'Snute', 11: 'Zest', 12: 'Maru', 13: 'Nerchio', 14: 'TaeJa', 15: 'NesTea', 16: 'Stephano', 17: 'Dark', 18: 'Neeb', 19: 'HerO', 20: 'Leenock', 21: 'Bomber', 22: 'DongRaeGu', 23: 'Stats', 24: 'HyuN', 25: 'Rain', 26: 'Solar', 27: 'Jaedong', 28: 'herO', 29: 'Classic', 30: 'MarineKing', 31: 'Scarlett', 32: 'aLive', 33: 'soO', 34: 'NaNiwa', 35: 'viOLet', 36: 'ShoWTimE', 37: 'MaNa', 38: 'Sen', 39: 'ForGG', 40: 'Dear', 41: 'Soulkey', 42: 'PuMa', 43: 'GuMiho', 44: 'HuK', 45: 'Hydra', 46: 'SpeCial', 47: 'TRUE', 48: 'Jim', 49: 'Elazer', 50: 'jjakji', 51: 'XiGua', 52: 'MacSed', 53: 'Patience', 54: 'StarDust', 55: 'San', 56: 'TooDming', 57: 'VortiX', 58: 'FruitDealer', 59: 'Creator', 60: 'YoDa', 61: 'ByuL', 62: 'ThorZaIN', 63: 'iAsonu', 64: 'Trap', 65: 'Rogue', 66: 'TLO', 67: 'Kelazhur', 68: 'Losira', 69: 'uThermal', 70: 'Symbol', 71: 'Genius', 72: 'Bly', 73: 'Lilbow', 74: 'Bunny', 7

In [45]:
def get_id(val):
    for key, value in player_dict.items():
        if val == value:
             return key

games = []
playerInfo = {}
with open('train-57922.csv', newline='') as csv_file: # replace csv file with the training dataset being used
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        player1 = get_id(str(row[1]))
        player2 = get_id(str(row[4]))
        
        if player1 in playerInfo.keys():
            playerInfo[player1][0] += 1
        else:
            # player1 is not in keys
            playerInfo[player1] = [1,0] # [number of games played, games won]
        
        if player2 is None:
            global players
            player_dict[players] = str(row[4])
            player2 = players
            players += 1
        if row[2] == "[loser]":
            score = -1
        else:
            score = +1
            playerInfo[player1][1] += 1
        games.append((player1, player2, score))
        
for player in playerInfo:
    playerInfo[player].append(playerInfo[player][1]/playerInfo[player][0])
#print(games) #it works

### Code for making shorter training csv files

##### Only run this once to create the smaller training dataset csv files

In [17]:
username_to_id = dict((v,k) for k,v in player_dict.items())

def get_dict_for_CSV(totalRowsLeft: int, debug: bool) -> dict:
    playerDivide = {}
    otrPlayerInfo = dict(playerInfo)
    playersRemaining = set()

    for i in range(988):
        playersRemaining.add(i)

    while len(playersRemaining) > 0 and totalRowsLeft > 0:
        divideGames = int(totalRowsLeft / len(playersRemaining))
        otrPlayersRemaining = playersRemaining.copy()
        
        if debug:
            print("==1== divideGames:", divideGames)
            print("==2== totalRowsLeft:", totalRowsLeft)
            print("==3== playersRemaining:", playersRemaining)
            print(">>> otrPlayersRemaining:", otrPlayersRemaining)
            
        if divideGames == 0:
            for player in otrPlayersRemaining:
                if totalRowsLeft > 0:
                    playerDivide[player] += 1 
                    totalRowsLeft -= 1
                else:
                    break

        for player in otrPlayersRemaining:
            if player not in playerDivide.keys():
                if otrPlayerInfo[player][0] <= divideGames:
                    playerDivide[player] = otrPlayerInfo[player][0]
                    playersRemaining.remove(player)
                    totalRowsLeft -= otrPlayerInfo[player][0]
                    otrPlayerInfo[player][0] = 0
                else:
                    playerDivide[player] = divideGames
                    totalRowsLeft -= divideGames
                    otrPlayerInfo[player][0] -= divideGames
            else:
                if otrPlayerInfo[player][0] <= divideGames:
                    playerDivide[player] += otrPlayerInfo[player][0]
                    playersRemaining.remove(player)
                    totalRowsLeft -= otrPlayerInfo[player][0]
                    otrPlayerInfo[player][0] = 0
                else:
                    playerDivide[player] += divideGames
                    totalRowsLeft -= divideGames
                    otrPlayerInfo[player][0] -= divideGames
                    
    return playerDivide

def gen_CSV(num_rows: int, debug: bool, old_csv: str, new_csv_name: str) -> None:
    
    player_divide = dict(get_dict_for_CSV(num_rows, debug))
    
    with open(old_csv, newline='') as old_csv:
        with open(new_csv_name, "w") as new_csv:
            csv_reader = csv.reader(old_csv, delimiter=',')
            for row in csv_reader:
                if sum(player_divide.values()) <= 0:
                    break
                
                the_id = username_to_id[str(row[1])]
                if player_divide[the_id] > 0:
                    new_csv.write(','.join(row))
                    new_csv.write('\n')
                    player_divide[the_id] -= 1

In [6]:
gen_CSV(173767, False, 'train.csv', 'train-173767.csv')

In [13]:
gen_CSV(154459, False, 'train.csv', 'train-154459.csv')

In [18]:
gen_CSV(57922 , False, 'train.csv', 'train-57922.csv')

### Win probability and graphical model

In [46]:
import time
start = time.time()

In [47]:
nplayers = max( [max(g[0],g[1]) for g in games] )+1
nlevels = 10
# we will also test scale values: .4, .5, .6
scale = .5     # this scales how skill difference translates to win probability

X = [None]*nplayers
for i in range(nplayers):
    X[i] = gm.Var(i, nlevels)   

#    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j                   
        Pwin[i,j] = (1./(1+np.exp(-scale*diff)))

# before any games, uniform belief over skill levels for each player:
factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

# Now add the information from each game:
for g in games:
    P1,P2,win = g[0],g[1],g[2]
    if P1 != P2:
        if P1>P2: P1,P2,win=P2,P1,-win
        inside = gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin)
        factors.append(inside)

In [48]:
model = gm.GraphModel(factors)
model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)
print("Finished here")

Finished here


In [49]:
if model.nvar < 0:       
    jt = model.joint()
    jt /= jt.sum()       
    bel = [jt.marginal([i]) for i in range(nplayers)] 
else:                    
    from pyGM.messagepass import LBP, NMF
    #lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
    lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field

Iter 0: -63002.199845799056
Iter 1: -34983.03481564578
Iter 2: -34106.28351996313
Iter 3: -33913.83113796054
Iter 4: -33858.54465952842
Iter 5: -33842.453006470685
Iter 6: -33836.450993821425
Iter 7: -33833.319188731504
Iter 8: -33831.39179655556
Iter 9: -33830.06941864464
Iter 10: -33829.050890138664


### Ranking players by predicted skill

In [50]:
print("Mean skill estimates: ")
print([ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)] )

Mean skill estimates: 
[8.351554477967648, 8.999888460482307, 8.999708530365503, 8.952479949818766, 8.999448852377693, 8.99999999999998, 7.947619762247287, 8.999999999999908, 8.99364133661515, 8.999992811316297, 8.99996036265476, 8.999950016584148, 8.998579849437778, 8.999929850298104, 8.960741761535937, 7.322966818442847, 7.3032804338674975, 8.999999779822346, 8.999999999965015, 7.818996125705687, 8.354878789592602, 8.534934556539335, 8.984678982003096, 8.999585961651606, 8.806886214181455, 8.994408819504004, 8.99999998650227, 8.990170475348467, 8.99999999879821, 8.971415707907218, 8.608263954764244, 8.825780094747513, 8.969973215342268, 8.999999987521901, 8.884053423663781, 8.005324942023552, 8.99910141063294, 7.217797974371831, 7.005077577832006, 8.999671226286006, 8.997882427412545, 8.89089505102769, 7.699270175235727, 8.999999999933655, 7.307222719457794, 8.999494227156083, 8.083644973709871, 8.980077431628722, 6.782589076994198, 8.280966809853393, 8.981247223546266, 6.39229335918

### Predicting match outcomes using our algorithm

In [51]:
outcomes = []
with open('valid.csv', newline='') as csv_file2:
    csv_reader2 = csv.reader(csv_file2, delimiter=',')
    for row in csv_reader2:
        player1 = get_id(str(row[1]))
        player2 = get_id(str(row[4]))
        if player2 is None:
            global players
            player_dict[players] = str(row[4])
            player2 = players
            players += 1
        i,j = player1, player2
        if i<j:
            win_percent = (bel[i]*bel[j]*gm.Factor([X[i],X[j]],Pwin)).table.sum()
            if win_percent < 0.5:
                outcomes.append(-1)
            else:
                outcomes.append(1)
        else:
            win_percent = (bel[i]*bel[j]*gm.Factor([X[i],X[j]],1-Pwin)).table.sum()
            if win_percent < 0.5:
                outcomes.append(-1)
            else:
                outcomes.append(1)
#print(outcomes)
end = time.time()
print("It took ", end - start, " seconds to complete.")

It took  253.39318108558655  seconds to complete.


In [52]:
correct = []
with open('valid.csv', newline='') as csv_file2:
    csv_reader2 = csv.reader(csv_file2, delimiter=',')
    for row in csv_reader2:
        if row[2] == "[loser]":
            correct.append(-1)
        else:
            correct.append(1)

In [53]:
right = 0
index = 0
for i in outcomes:
    if outcomes[index] == correct[index]:
        right+=1
    index+=1
print(right)
dec = int(right)/94007
percentage = "{:.2%}".format(dec)

63442


In [54]:
print("Our algorithm estimated the correct outcome ", percentage, " of the time.")

Our algorithm estimated the correct outcome  67.49%  of the time.


### Predicting match outcomes using simple method

In [82]:
outcomes = []
with open('valid.csv', newline='') as csv_file2:
    csv_reader2 = csv.reader(csv_file2, delimiter=',')
    for row in csv_reader2:
        player1 = get_id(str(row[1]))
        player2 = get_id(str(row[4]))
        if player2 is None:
            global players
            player_dict[players] = str(row[4])
            player2 = players
            players += 1
        i = playerInfo.get(player1)[2]
        j = 0
        if playerInfo.get(player2) is not None:
            j = playerInfo.get(player2)[2]
        if i > j:
            outcomes.append(1)
        else:
            outcomes.append(-1)

In [83]:
right = 0
index = 0
for i in outcomes:
    if outcomes[index] == correct[index]:
        right+=1
    index+=1
print(right)
dec = int(right)/94007
percentage = "{:.2%}".format(dec)

61868


In [84]:
print("The simple approach estimated the correct outcome ", percentage, " of the time.")

The simple approach estimated the correct outcome  65.81%  of the time.
